In [1]:
from pathlib import Path
import sys
import pandas as pd
import numpy as np
import pickle
import sklearn
from datetime import datetime
from sklearn.tree import DecisionTreeClassifier
from modAL.models import ActiveLearner
import torch

package_path = Path.cwd().parent
if package_path not in sys.path:
    sys.path.append(str(package_path))

from utils.dataset_class import Phase3DataSet, Setting
from custom_pipeline import get_model_pkl, dump_model_pkl
from models.meta.platipus_class import Platipus
from hpc_scripts.param_generator import get_all_params
from models.meta.init_params import init_params

In [2]:
x_stateset_set0 = np.genfromtxt(f'./statesets/scaled_stateset_{datetime.today().date()}.csv', delimiter=',')
x_stateset_set1 = np.genfromtxt(f'./statesets/scaled_stateset_{datetime.today().date()}.csv', delimiter=',')

rvol_raw_set0 = pd.read_csv('./extra_data/Me2NH2I_10uL_stateset.link.csv')
rvol_raw_set1 = pd.read_csv('./extra_data/Me2NH2I_10uL_stateset.link.csv')

ss_reagent_vols_set0 = rvol_raw_set0[[col for col in rvol_raw_set0.columns if 'Reagent' in col]]
ss_reagent_vols_set1 = rvol_raw_set1[[col for col in rvol_raw_set1.columns if 'Reagent' in col]]

In [6]:
"""
query_indices_set0 = [10022, 1441]
query_indices_set1 = [17833, 9275]
query_instances_set0 = []
query_instances_set1 = []
for idx in query_indices_set0:
    query_instances_set0.append(x_stateset_set0[idx])
for idx in query_indices_set1:
    query_instances_set1.append(x_stateset_set0[idx])
"""

In [3]:
all_params = get_all_params(cross_validate=False)
params = all_params[172]
amine = ['JMXLWMIFDJCGBV-UHFFFAOYSA-N']
train_params_set0 = init_params(params)
train_params_set0['gpu_id'] = 0

train_params_set1 = init_params(params)
train_params_set1['gpu_id'] = 1

platipus_set0 = Platipus(train_params_set0, amine=amine,
                    model_name=train_params_set0['model_name'],
                    epoch_al=True)
platipus_set0.load_model('./plat_trained_model/testing/drp_chem_10shot_4000.pkl')
platipus_set0.current_epoch = 4000

platipus_set1 = Platipus(train_params_set1, amine=amine,
                    model_name=train_params_set1['model_name'],
                    epoch_al=True)
platipus_set1.load_model('./plat_trained_model/testing/drp_chem_10shot_4000.pkl')
platipus_set1.current_epoch = 4000

device_set0=torch.device("cuda:0")
device_set1=torch.device("cuda:1")

"""

platipus = Platipus(train_params, amine=amine,
                    model_name=train_params['model_name'],
                    epoch_al=True)
platipus.load_model('./plat_trained_model/testing/drp_chem_10shot_4000.pkl')
platipus.current_epoch = 4000
device=torch.device("cuda:1")
"""


Meta learning rate = 0.001
Getting FULL dataset
Holding out ['CALQKRVFTWDYDG-UHFFFAOYSA-N', 'KOAGKPNEVYEZDU-UHFFFAOYSA-N', 'FCTHQYIDLRRROX-UHFFFAOYSA-N', 'JMXLWMIFDJCGBV-UHFFFAOYSA-N']
Number of reactions in training set 1477
Number of successful reactions in the training set 215
Generating training batches
Generating testing batches for testing! DO NOT RUN IF YOU SEE THIS LINE!
Meta learning rate = 0.001
Getting FULL dataset
Holding out ['CALQKRVFTWDYDG-UHFFFAOYSA-N', 'KOAGKPNEVYEZDU-UHFFFAOYSA-N', 'FCTHQYIDLRRROX-UHFFFAOYSA-N', 'JMXLWMIFDJCGBV-UHFFFAOYSA-N']
Number of reactions in training set 1477
Number of successful reactions in the training set 215
Generating training batches
Generating testing batches for testing! DO NOT RUN IF YOU SEE THIS LINE!


'\n\nplatipus = Platipus(train_params, amine=amine,\n                    model_name=train_params[\'model_name\'],\n                    epoch_al=True)\nplatipus.load_model(\'./plat_trained_model/testing/drp_chem_10shot_4000.pkl\')\nplatipus.current_epoch = 4000\ndevice=torch.device("cuda:1")\n'

In [4]:
set_id = 0 
iters, all_data, all_labels, x_t, y_t, x_v, y_v = platipus_set0.setup_active_learning(set_id, 'JMXLWMIFDJCGBV-UHFFFAOYSA-N')

In [5]:
xss_torch0 = torch.from_numpy(x_stateset_set0[:10000]).float().to(device_set0)
xss_torch1 = torch.from_numpy(x_stateset_set0[10000:]).float().to(device_set0)

In [6]:
results = {
    10022:1,
    1441:0,
    6127:0,
}
for exp_num in results:
    if exp_num >= 10000:
        x_t = torch.cat((x_t, xss_torch1[exp_num-10000].view(1, 50)))
        y_t = torch.cat((y_t, torch.from_numpy(np.array([results[exp_num]])).to(device_set0)))
    else:
        x_t = torch.cat((x_t, xss_torch1[exp_num].view(1, 50)))
        y_t = torch.cat((y_t, torch.from_numpy(np.array([results[exp_num]])).to(device_set0)))

In [9]:
#x_t = torch.cat((x_t, xss_torch0[7833].view(1, 50)))
y_t = torch.cat((y_t, torch.from_numpy(np.array([0])).to(device_set0)))

In [7]:
value0, index0 = platipus_set0.phase3_active_learning(x_t, y_t, xss_torch0)

In [8]:
print(value0, index0)

tensor(0.5261, device='cuda:0', grad_fn=<MinBackward0>) tensor(9716, device='cuda:0')


In [9]:
del value0
del index0
torch.cuda.empty_cache()

In [6]:
del xss_torch0
torch.cuda.empty_cache()

In [10]:
#xss_torch1 = torch.from_numpy(x_stateset_set0[10000:]).float().to(device_set0)
value1, index1 = platipus_set0.phase3_active_learning(x_t, y_t, xss_torch1)

In [11]:
print(value1, index1)
del value1
torch.cuda.empty_cache()

tensor(nan, device='cuda:0', grad_fn=<MinBackward0>) tensor(0, device='cuda:0')


In [12]:
ss_reagent_vols_set0.iloc[9716]

Reagent1 (ul)    358
Reagent2 (ul)     10
Reagent3 (ul)     32
Reagent4 (ul)      0
Reagent5 (ul)      0
Reagent6 (ul)      0
Reagent7 (ul)    100
Reagent8 (ul)      0
Reagent9 (ul)      0
Name: 9716, dtype: int64

In [9]:
#value, index = platipus.phase3_active_learning(x_t, y_t, xss_torch)
value, index

(tensor(0.5000, device='cuda:1', grad_fn=<MinBackward0>),
 tensor(7833, device='cuda:1'))

In [7]:
x_t = torch.cat((x_t, xss_torch[7833].view(1, 50)))

In [8]:
y_t = torch.cat((y_t, torch.from_numpy(np.array([0])).to(device)))

In [9]:
y_t.shape

torch.Size([11])

In [10]:
value, index = platipus.phase3_active_learning(x_t, y_t, xss_torch)

In [11]:
print(value, index)

tensor(0.5556, device='cuda:1', grad_fn=<MinBackward0>) tensor(9275, device='cuda:1')


In [16]:
set_id = 1 
iters, all_data, all_labels, x_t, y_t, x_v, y_v = platipus_set1.setup_active_learning(set_id, 'JMXLWMIFDJCGBV-UHFFFAOYSA-N')

In [17]:
xss_torch0 = torch.from_numpy(x_stateset_set1[:10000]).float().to(device_set1)
value0, index0 = platipus_set1.phase3_active_learning(x_t, y_t, xss_torch0)

In [18]:
print(value0, index0)
del value0
torch.cuda.empty_cache()

tensor(0.5000, device='cuda:1', grad_fn=<MinBackward0>) tensor(6470, device='cuda:1')


In [19]:
xss_torch1 = torch.from_numpy(x_stateset_set1[10000:]).float().to(device_set1)
value1, index1 = platipus_set1.phase3_active_learning(x_t, y_t, xss_torch1)

In [20]:
print(value1, index1)
del value1
torch.cuda.empty_cache()

tensor(0.5000, device='cuda:1', grad_fn=<MinBackward0>) tensor(6125, device='cuda:1')


In [21]:
ss_reagent_vols_set0.iloc[6470]

Reagent1 (ul)    161
Reagent2 (ul)    100
Reagent3 (ul)     99
Reagent4 (ul)      0
Reagent5 (ul)      0
Reagent6 (ul)      0
Reagent7 (ul)    140
Reagent8 (ul)      0
Reagent9 (ul)      0
Name: 6470, dtype: int64

## PLATIPUS SET 1

In [13]:
set_id = 1
iters, all_data, all_labels, x_t, y_t, x_v, y_v = platipus_set1.setup_active_learning(set_id, 'JMXLWMIFDJCGBV-UHFFFAOYSA-N')

In [14]:
xss_torch0 = torch.from_numpy(x_stateset_set0[:10000]).float().to(device_set1)
xss_torch1 = torch.from_numpy(x_stateset_set0[10000:]).float().to(device_set1)

In [15]:
results = {
    17833:0,
    9275:0,
    6470:0,
}
for exp_num in results:
    if exp_num >= 10000:
        x_t = torch.cat((x_t, xss_torch1[exp_num-10000].view(1, 50)))
        y_t = torch.cat((y_t, torch.from_numpy(np.array([results[exp_num]])).to(device_set1)))
    else:
        x_t = torch.cat((x_t, xss_torch1[exp_num].view(1, 50)))
        y_t = torch.cat((y_t, torch.from_numpy(np.array([results[exp_num]])).to(device_set1)))

In [16]:
value0, index0 = platipus_set1.phase3_active_learning(x_t, y_t, xss_torch0)
print(value0, index0)
del value0
del index0
torch.cuda.empty_cache()

tensor(0.5018, device='cuda:1', grad_fn=<MinBackward0>) tensor(2378, device='cuda:1')


In [17]:
value1, index1 = platipus_set1.phase3_active_learning(x_t, y_t, xss_torch1)
print(value1, index1)
del value1
torch.cuda.empty_cache()

tensor(0.6820, device='cuda:1', grad_fn=<MinBackward0>) tensor(589, device='cuda:1')


In [18]:
ss_reagent_vols_set0.iloc[2378]

Reagent1 (ul)      6
Reagent2 (ul)    280
Reagent3 (ul)      4
Reagent4 (ul)      0
Reagent5 (ul)      0
Reagent6 (ul)      0
Reagent7 (ul)    210
Reagent8 (ul)      0
Reagent9 (ul)      0
Name: 2378, dtype: int64